In [1]:
import sys
from strategies import *
from bidder import Bidder
%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append(r'c:\users\ggoren\PycharmProjects\AGTProject')

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:

auction_model_id = "Qwen/Qwen2-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    auction_model_id,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(auction_model_id)


c:\Users\mayha\OneDrive\Desktop\year 4\algs and incentives\LLMTokenAuctions\.venv\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def apply_chat_template(messages, tokens):
    return f'<|im_start|>system\n{messages[0]["content"]}.<|im_end|>\n<|im_start|>user\n{messages[1]["content"]}<|im_end|>\n<|im_start|>assistant\n{"".join(tokens)}'

def get_next_word_probs(messages, tokens):
    text = apply_chat_template(messages, tokens)
    model_inputs = tokenizer([text], return_tensors="pt")

    outputs = model(model_inputs.input_ids)
    logits = outputs.logits

    next_token_logits = logits[:, -1, :]  # Get the logits for the last token
    next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

    # Get the top 100 most probable tokens
    top_k = 100
    top_probs, top_indices = torch.topk(next_token_probs, top_k, dim=-1)
    return top_probs, top_indices


In [7]:
 import numpy as np
 def q_integral(p, b,t,i,value):
    P = (p.T @ b)[t] - p[i][t] * b[i]
    S = b.sum() - b[i]
    return p[i][t] * value + (P - p[i][t] * S) * torch.log(torch.abs(value + S))

def token_payment(p, b, t, i, q):
    return 1/2 * ((q[t] - p[i][t]) * b[i] - (q_integral(p, b, t, i, b[i]) - q_integral(p, b,t, i, 0) - p[i][t] *b[i]))

def update_payments(payments, p, b, q):
    """
    Iterate every bidder and token to get its payments from the integral
    :param payments: Current payment vector to update
    :param p: Current bidder's distributions over token
    :param b: Bidders' bid
    :param q: Distribution aggregation
    :return: Updated payments vector
    """
    for i in range(b.shape[0]):
        for t in range(p.shape[0]):
            payment = token_payment(p, b, t, i, q).item()
            payments[i] += max(payment, 0)
    return payments


In [8]:
def qkl(b, combined_probs):
    # Ensure both tensors are float type
    b = b.float()  # Convert b to float if it's not already
    combined_probs = combined_probs.float()  # Convert combined_probs to float

    # Perform the matrix operation
    result = combined_probs.T @ b / b.sum()
    return result


def get_missing_tokens(tensor_a, tensor_b):
    indices_a_not_in_b_mask = ~torch.isin(tensor_a, tensor_b)
    indices_a_not_in_b = tensor_a[indices_a_not_in_b_mask].reshape(1, -1)
    return indices_a_not_in_b

def add_zero_probs_for_new_tokens(indices, probs, new_tokens):

    indices = torch.cat([indices, new_tokens], dim=1)
    zero_probs = torch.zeros_like(new_tokens)
    probs = torch.cat([probs, zero_probs], dim=1)
    return indices, probs

def sort_tokens(indices, probs):
    sorted_indices, sorted_order = torch.sort(indices)
    sorted_probs = probs[0, sorted_order]
    return sorted_indices, sorted_probs

def sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta):
    beta_missing_tokens = get_missing_tokens(top_indices_alpha, top_indices_beta)
    alpha_missing_tokens = get_missing_tokens(top_indices_beta, top_indices_alpha)

    full_alpha_indices, full_alpha_probs = add_zero_probs_for_new_tokens(top_indices_alpha, top_probs_alpha, alpha_missing_tokens)
    full_beta_indices, full_beta_probs = add_zero_probs_for_new_tokens(top_indices_beta, top_probs_beta, beta_missing_tokens)

    sorted_alpha_indices, sorted_alpha_probs = sort_tokens(full_alpha_indices, full_alpha_probs)
    sorted_beta_indices, sorted_beta_probs = sort_tokens(full_beta_indices, full_beta_probs)

    combined_indices = torch.cat([sorted_alpha_indices, sorted_beta_indices])
    combined_probs = torch.cat([sorted_alpha_probs, sorted_beta_probs])
    return combined_indices, combined_probs


def get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments, temperature=0.06):
    """
    This function receives distributions from both bidders, aggregates them into one distribution (q), calculates the costs
    and samples the next token
    :param b: Bidders' bid
    :param top_probs_alpha: Distribution of Bidder alpha over tokens
    :param top_indices_alpha: Mapping of indices to actual tokens for bidder alpha
    :param top_probs_beta: Distribution of Bidder beta over tokens
    :param top_indices_beta: Mapping of indices to actual tokens for bidder beta
    :param payments: Vector of payments
    :param temperature: Temperature parameter to control token sampling
    :return: The next sampled token and induced payments.
    """
    # combined_probs is p: A tensor for each bidder with dist over tokens
    # combined_indices is the actual token
    combined_indices, combined_probs = sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta)
    token_logits = qkl(combined_probs, b)
    payments = update_payments(payments, combined_probs, b, token_logits)
    token_temperature_logits = token_logits / temperature
    token_dist = torch.softmax(token_temperature_logits, dim=0)
    token = tokenizer.decode(combined_indices[0,torch.multinomial(token_dist, 1)].item())
    return token, payments


In [9]:
def get_token_probs_from_distributions(chosen_token, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta):
    """
    This function extracts the probability of a chosen token from the given distributions of both bidders.
    If the token is found in either distribution, return the probability; otherwise, return 0.
    
    :param chosen_token: The token for which we want to extract the probability.
    :param top_probs_alpha: Distribution of Bidder alpha over tokens.
    :param top_indices_alpha: Mapping of indices to actual tokens for Bidder alpha.
    :param top_probs_beta: Distribution of Bidder beta over tokens.
    :param top_indices_beta: Mapping of indices to actual tokens for Bidder beta.
    :return: The probability of the chosen token from both bidders.
    """
    
    # Token probability starts at 0
    token_prob_alpha = 0.0
    token_prob_beta = 0.0

    # Convert the chosen token to its token ID using the tokenizer
    chosen_token_id = tokenizer.encode(chosen_token, add_special_tokens=False)[0]

    # Check if the token exists in top_indices_alpha
    if chosen_token_id in top_indices_alpha:
        # Get the index of the chosen token in top_indices_alpha
        index_alpha = (top_indices_alpha == chosen_token_id).nonzero(as_tuple=True)[1]
        # Add the corresponding probability from top_probs_alpha
        token_prob_alpha += top_probs_alpha[0, index_alpha].item()

    # Check if the token exists in top_indices_beta
    if chosen_token_id in top_indices_beta:
        # Get the index of the chosen token in top_indices_beta
        index_beta = (top_indices_beta == chosen_token_id).nonzero(as_tuple=True)[1]
        # Add the corresponding probability from top_probs_beta
        token_prob_beta += top_probs_beta[0, index_beta].item()

    return token_prob_alpha, token_prob_beta


In [10]:

messages_alpha = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair. Do not include an intro, only write an ad."}, {"role":"user", "content":"Write a two-sentence ad for a flight to Hawaii using Alpha Airlines. Ensure Alpha Airlines is at the beginning of each sentence."}
]
messages_beta = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair, Do not include an intro, only write an ad."}, {"role":"user", "content":"Write a two-sentence ad for a vacation in Hawaii at the Beta Resort. Ensure Beta Resort is at the beginning of each sentence."}
]

In [21]:
messages_alpha_competing = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair, Do not include an intro, only write an ad."}, {"role":"user", "content":"Write a two-sentence ad for a vacation in Hawaii at the Alpha Resort. Ensure Alpha Resort is at the beginning of each sentence."}
]

In [11]:
import time
def generate_text(alpha_airlines_bidder: Bidder, beta_resort_bidder: Bidder, sentence_limit=2, token_count=100, verbose = True):
    start_time = time.time()
    sentence_counter = 0
    tokens = []
    alpha_airlines_bid = alpha_airlines_bidder.get_bid("".join(tokens))
    beta_resort_bid = beta_resort_bidder.get_bid("".join(tokens))
    b = torch.tensor([alpha_airlines_bid, beta_resort_bid])
    payments = np.zeros_like(b)
    chosen_tokens_probs_result_alpha = []
    chosen_tokens_probs_result_beta = []
    chosen_tokens_payment_prob_ratio_result_alpha = []
    chosen_tokens_payment_prob_ratio_result_beta = []
    for _ in range(token_count):
        top_probs_alpha, top_indices_alpha  = get_next_word_probs(alpha_airlines_bidder.messages_alpha, tokens)
        top_probs_beta, top_indices_beta = get_next_word_probs(beta_resort_bidder.messages_beta, tokens)
        next_token, payments = get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments)

        if verbose:
            print(next_token, end='')

        if next_token == "<|im_end|>" or "<|" in next_token or "|>" in next_token:
                break

        tokens.append(next_token)
        chosen_token_probs_alpha, chosen_token_probs_beta= get_token_probs_from_distributions(next_token,top_probs_alpha=top_probs_alpha,top_indices_alpha=top_indices_alpha, top_probs_beta= top_probs_beta, top_indices_beta=top_indices_beta)
        chosen_tokens_probs_result_alpha.append(chosen_token_probs_alpha)
        chosen_tokens_probs_result_beta.append(chosen_token_probs_beta)
        chosen_tokens_payment_prob_ratio_result_alpha.append(chosen_token_probs_alpha/ float(payments[0]))
        chosen_tokens_payment_prob_ratio_result_beta.append(chosen_token_probs_beta/ float(payments[1]))

        # Break after 2 sentences
        if "." in next_token or "!" in next_token or "?" in next_token:
            sentence_counter += 1
            if sentence_counter >= sentence_limit:
                break
        alpha_airlines_bid = alpha_airlines_bidder.get_bid("".join(tokens)) if payments[0] <= alpha_airlines_bidder.budget else 0.01
        beta_resort_bid = beta_resort_bidder.get_bid("".join(tokens)) if payments[1] <= beta_resort_bidder.budget else 0.01
        b = torch.tensor([alpha_airlines_bid, beta_resort_bid])
    if verbose:
        print("\n", (time.time() - start_time) / 60, "minutes")

    return "".join(tokens), payments ,chosen_tokens_probs_result_alpha, chosen_tokens_probs_result_beta, chosen_tokens_payment_prob_ratio_result_alpha, chosen_tokens_payment_prob_ratio_result_beta

In [12]:
company_names = ['Alpha Airlines', 'Beta Resort']

In [13]:
def string_in_string(substring, main_string):
    lower_main = main_string.lower()
    lower_sub = substring.lower()
    index = lower_main.find(lower_sub)
    main_length = len(main_string)
    
    if index != -1:
        return (main_length - index) / main_length  # Higher score for lower index, normalized by length
    else:
        return 0


In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a new instruction-following model, like flan-T5
model_name = "google/flan-t5-small"  # Try flan-t5 models
model_eval = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer_eval = AutoTokenizer.from_pretrained(model_name)
    

In [15]:
import openai
import os
import torch
import re

def evaluate_outcome_using_llm(company_name, outcome, useOpenAI = True, openai_model_name='gpt-4o-mini', verbose=True):
    if useOpenAI:
        api_key = os.getenv("OPENAI_API_KEY")

        if not api_key:
            raise ValueError("OpenAI API key is not set. Please set the OPENAI_API_KEY environment variable.")

        openai.api_key = api_key

        eval_prompt = f"""
        You are an expert ad evaluator. Evaluate the quality of the following ad text for the company "{company_name}" based on relevance, coherence, and clarity.
        It's very important that the companty's name "{company_name}" will be a part of the ad.

        Ad Text:
        {outcome}

        Provide a single score between 1 and 10 that reflects the overall quality of the ad. Respond with **only the number** and nothing else.
        """


        response = openai.ChatCompletion.create(
            model=openai_model_name,
            messages=[
                {"role": "system", "content": "You are a text evaluation assistant."},
                {"role": "user", "content": eval_prompt}
            ],
            max_tokens=2,  
            temperature=0  
        )

        response_text = response['choices'][0]['message']['content'].strip()
    else:
        eval_prompt = f"""
        Evaluate the following ad text for the company: "{company_name}".

        Ad Text:
        {outcome}

        Provide **only** a number between 1 and 10. Do not provide any explanation or additional text. Just the number.
        """

        # Tokenize the evaluation prompt
        inputs = tokenizer_eval(eval_prompt, return_tensors="pt").to(model_eval.device)

        # Generate a response with a stricter token limit to prevent extra output
        with torch.no_grad():
            outputs = model_eval.generate(inputs["input_ids"], matuvx_new_tokens=1000)  # Further limiting tokens to just 2

        # Decode the response
        response_text = tokenizer_eval.decode(outputs[0], skip_special_tokens=True).strip()

    if verbose:
        print("LLM Response:\n", response_text)

    match = re.match(r"^\d+$", response_text) 

    if match:
        score = int(match.group(0))
        if 1 <= score <= 10:
            return score

    print("No valid score found in the response.")
    return -1


In [16]:
# Example usage:
comany_name = "Beta Resort"
outcome = "At the heart of paradise, the Beta Resort awaits your arrival. Here, the Alpha Airlines flight takes"
score = evaluate_outcome_using_llm(comany_name, outcome)
print(f"Evaluation Score: {score}")


LLM Response:
 5
Evaluation Score: 5


In [17]:
import json
from datetime import datetime

# Helper function to calculate averages
def calculate_averages(total_values, num_tries):
    return {key: value / num_tries for key, value in total_values.items()}

# Helper function to build individual result dictionary
def build_result_dict(name_appeared, avg_prob, score, payment, outcome, outcome_length, llm_evaluation_score, num_of_tokens, avg_token_payment_prob_ratio):
    return {
        "outcome": outcome,
        "outcome_length": outcome_length,
        "num_of_tokens": num_of_tokens,
        "name_appeared": name_appeared,
        "avg_chosen_token_probability": avg_prob,
        "name_appearance_score": score,
        "llm_evaluation_score": llm_evaluation_score,
        "payment": payment,
        "avg_payment_per_token": payment / num_of_tokens,
        "avg_token_payment_prob_ratio": avg_token_payment_prob_ratio
    }

# Helper function to accumulate totals for averages
def accumulate_totals(totals, values):
    for key, value in values.items():
        totals[key] += value

def run_exp(num_tries: int, alpha_airlines_bidder: Bidder, beta_resort_bidder: Bidder, fileprefix: str, verbose=True):
    result_alpha = []
    result_beta = []

    # Initialize totals for averages
    totals_alpha = {"name_appeared": 0, "avg_prob": 0, "name_appear_score": 0, "payment": 0, "llm_score": 0, "avg_pay_token": 0, "avg_pay_prob_ratio": 0}
    totals_beta = {"name_appeared": 0, "avg_prob": 0, "name_appear_score": 0, "payment": 0, "llm_score": 0, "avg_pay_token": 0,  "avg_pay_prob_ratio": 0}
    total_outcome_length = 0
    total_num_of_tokens = 0
    openai_model_name = 'gpt-4o-mini'

    for i in range(num_tries):
        outcome, ad_payments, chosen_tokens_probs_result_alpha, chosen_tokens_probs_result_beta, chosen_tokens_payment_prob_ratio_result_alpha, chosen_tokens_payment_prob_ratio_result_beta = generate_text(
            alpha_airlines_bidder=alpha_airlines_bidder, beta_resort_bidder=beta_resort_bidder)
        
        name_appear_alpha_score = string_in_string(substring=alpha_airlines_bidder.bidder_company, main_string=outcome)
        name_appear_beta_score = string_in_string(substring=beta_resort_bidder.bidder_company, main_string=outcome)
        
        is_name_appear_alpha = 0 if name_appear_alpha_score == 0 else 1
        is_name_appear_beta = 0 if name_appear_beta_score == 0 else 1
        
        num_of_tokens = len(chosen_tokens_probs_result_alpha)
        
        avg_chosen_token_prob_alpha = sum(chosen_tokens_probs_result_alpha) / num_of_tokens
        avg_chosen_token_prob_beta = sum(chosen_tokens_probs_result_beta) / num_of_tokens
        
        avg_token_payment_prob_ratio_alpha = sum(chosen_tokens_payment_prob_ratio_result_alpha) / num_of_tokens
        avg_token_payment_prob_ratio_beta = sum(chosen_tokens_payment_prob_ratio_result_beta) / num_of_tokens


        llm_evaluation_score_alpha = evaluate_outcome_using_llm(alpha_airlines_bidder.bidder_company, outcome, openai_model_name=openai_model_name, verbose=verbose)
        llm_evaluation_score_beta = evaluate_outcome_using_llm(beta_resort_bidder.bidder_company, outcome, openai_model_name=openai_model_name, verbose=verbose)

        payment_alpha =  float(ad_payments[0])
        payment_beta =  float(ad_payments[1])
        
        outcome_length = len(outcome)
        total_outcome_length += outcome_length
        total_num_of_tokens += num_of_tokens

        # Append individual results
        result_alpha.append(build_result_dict(is_name_appear_alpha, avg_chosen_token_prob_alpha, name_appear_alpha_score, payment_alpha, outcome, outcome_length, llm_evaluation_score_alpha, num_of_tokens, avg_token_payment_prob_ratio_alpha))
        result_beta.append(build_result_dict(is_name_appear_beta, avg_chosen_token_prob_beta, name_appear_beta_score, payment_beta, outcome, outcome_length, llm_evaluation_score_beta, num_of_tokens, avg_token_payment_prob_ratio_beta))
        

        # Accumulate totals for Alpha and Beta
        accumulate_totals(totals_alpha, {"name_appeared": is_name_appear_alpha, "avg_prob": avg_chosen_token_prob_alpha, "name_appear_score": name_appear_alpha_score, "payment": payment_alpha, "llm_score": llm_evaluation_score_alpha, "avg_pay_token": payment_alpha/num_of_tokens, "avg_pay_prob_ratio": avg_token_payment_prob_ratio_alpha})
        accumulate_totals(totals_beta, {"name_appeared": is_name_appear_beta, "avg_prob": avg_chosen_token_prob_beta, "name_appear_score": name_appear_beta_score, "payment": payment_beta, "llm_score": llm_evaluation_score_beta, "avg_pay_token": payment_beta/num_of_tokens, "avg_pay_prob_ratio": avg_token_payment_prob_ratio_beta})

    # Calculate averages
    avg_alpha = calculate_averages(totals_alpha, num_tries)
    avg_beta = calculate_averages(totals_beta, num_tries)
    avg_outcome_length = total_outcome_length / num_tries
    avg_num_of_tokens = total_num_of_tokens / num_tries

    # save input parameters
    input_alpha = {"bid": alpha_airlines_bidder.bid, "strategy": alpha_airlines_bidder.strategy_func.__name__,
                   "budget": alpha_airlines_bidder.budget, "company_name": alpha_airlines_bidder.bidder_company}
    input_beta = {"bid": beta_resort_bidder.bid, "strategy": beta_resort_bidder.strategy_func.__name__,
                   "budget": beta_resort_bidder.budget, "company_name": beta_resort_bidder.bidder_company}

    # Structuring results in a JSON-compatible dictionary
    results = {
        "Alpha Airlines": {
            "input": input_alpha,
            "individual_results": result_alpha,
            "averages": avg_alpha
        },
        "Beta Resort": {
            "input": input_beta,
            "individual_results": result_beta,
            "averages": avg_beta
        },
        "averages": {
            "avg_outcome_length": avg_outcome_length,
            "avg_num_of_tokens": avg_num_of_tokens
        },
        'auction_model': auction_model_id,
        'eval_model': openai_model_name
    }

    # Generating output file name
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Current date and time
    output_file = f"{fileprefix}_experiment_results_{num_tries}_tries_{timestamp}.json"

    # Writing the results into a JSON file
    with open(output_file, 'w') as file:
        json.dump(results, file, indent=4)

    return result_alpha, result_beta, output_file


In [45]:
alpha_airlines_bidder = Bidder(2.0, decreasing_based_on_position_strategy, 'Alpha Airlines', budget=12)
beta_resort_bidder = Bidder(3.0, decreasing_based_on_position_strategy, 'Beta Resort', budget=12)
num_tries = 1

results_3_2_alpha, results_3_2_beta, filename = run_exp(
    num_tries=num_tries, 
    alpha_airlines_bidder=alpha_airlines_bidder, 
    beta_resort_bidder=beta_resort_bidder,
    fileprefix="decreasing_based_on_position_strategy"
)


C:\Users\mayha\AppData\Local\Temp\ipykernel_28852\1190652271.py:7: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3641.)
  result = combined_probs.T @ b / b.sum()


Experience the ultimate

KeyboardInterrupt: 

In [51]:
alpha_airlines_bidder = Bidder(2.0, naive_strategy, 'Alpha Airlines', budget=12)
beta_resort_bidder = Bidder(3.0, naive_strategy, 'Beta Resort', budget=12)
num_tries = 2

results_3_2_alpha, results_3_2_beta, filename = run_exp(
    num_tries=num_tries, 
    alpha_airlines_bidder=alpha_airlines_bidder, 
    beta_resort_bidder=beta_resort_bidder,
    fileprefix="naive"
)


At the heart of paradise, the Beta Resort awaits your arrival. Dive into the crystal-clear waters of Hawaii's azure seas, where Alpha Airlines helms your journey to the ultimate destination.
 11.21407786210378 minutes
LLM Response:
 6
LLM Response:
 7
At the heart of paradise, the Beta Resort awaits your arrival. Here, the Alpha Airlines flight to Hawaii takes you to the heart of paradise, where the Beta Resort awaits your arrival.
 9.691122070948284 minutes
LLM Response:
 6
LLM Response:
 7


In [ ]:
results_3_2_alpha

In [ ]:
results_3_2_beta

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, name_contest_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, name_first_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(2.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, name_contest_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, name_first_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort', budget=12), verbose=True)
print(ad_payments)

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(3.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

In [19]:
strategies = [naive_strategy, name_contest_strategy, name_first_strategy, decreasing_based_on_position_strategy]

In [ ]:
counter = 0
experiment_count = len(strategies) * 2
for alpha_strategy in strategies:
    for alpha_bid in [3.0]:
        for beta_bid in [3.0]:
            for beta_strategy in [naive_strategy]:
                for budget in [8, 1000]:
                    alpha_airlines_bidder = Bidder(alpha_bid, alpha_strategy, 'Alpha Airlines', budget=budget)
                    beta_resort_bidder = Bidder(beta_bid, beta_strategy, 'Beta Resort', budget=budget)
                    num_tries = 20

                    results_alpha, results_beta, filename = run_exp(
                        num_tries=num_tries,
                        alpha_airlines_bidder=alpha_airlines_bidder,
                        beta_resort_bidder=beta_resort_bidder,
                        fileprefix=f"{alpha_strategy.__name__}_vs_{beta_strategy.__name__}_budget_{budget}_bid_{alpha_bid}")
                    counter += 1
                    print(f"Finished experiment {counter}/{experiment_count} with setting {alpha_strategy.__name__},{alpha_bid},{beta_bid},{beta_strategy.__name__},{budget}")

In [ ]:
outcome, ad_payments = generate_text(alpha_airlines_bidder=Bidder(4.0, naive_strategy, 'Alpha Airlines', budget=12),
                                     beta_resort_bidder=Bidder(3.0, naive_strategy, 'Beta Resort',budget=12), verbose=True)
print(ad_payments)

In [ ]:
# running the exp for competing companys same budget diffrent stategies
counter = 0
budget = 1000
experiment_count = len(strategies) * len(strategies)
for alpha_strategy in strategies:
    for alpha_bid in [3.0]:
        for beta_bid in [3.0]:
            for beta_strategy in strategies:
                company_names = ['Alpha Resort', 'Beta Resort']
                alpha_airlines_bidder = Bidder(alpha_bid, alpha_strategy, 'Alpha Resort', budget=budget, company_names = company_names, messages_alpha=messages_alpha_competing)
                beta_resort_bidder = Bidder(beta_bid, beta_strategy, 'Beta Resort', budget=budget, company_names=company_names, messages_alpha=messages_alpha_competing)
                num_tries = 2

                results_alpha, results_beta, filename = run_exp(
                    num_tries=num_tries,
                    alpha_airlines_bidder=alpha_airlines_bidder,
                    beta_resort_bidder=beta_resort_bidder,
                    fileprefix=f"{alpha_strategy.__name__}_vs_{beta_strategy.__name__}__competing_companies_same_budget")
                counter += 1
                print(f"Finished experiment {counter}/{experiment_count} with setting {alpha_strategy.__name__},{alpha_bid},{beta_bid},{beta_strategy.__name__},{budget}")

Experience the ultimate in relaxation and adventure at the

In [ ]:
# running the exp for competing companys
counter = 0
experiment_count = len(strategies) * len(strategies)
alpha_budget = 12
for alpha_strategy in strategies:
    for alpha_bid in [3.0]:
        for beta_bid in [3.0]:
            for beta_budget in [8, 12,1000]:
                company_names = ['Alpha Resort', 'Beta Resort']
                alpha_airlines_bidder = Bidder(alpha_bid, alpha_strategy, 'Alpha Resort', budget=alpha_budget, company_names = company_names, messages_alpha=messages_alpha_competing)
                beta_resort_bidder = Bidder(beta_bid, alpha_strategy, 'Beta Resort', budget=beta_budget, company_names=company_names, messages_alpha=messages_alpha_competing)
                num_tries = 20

                results_alpha, results_beta, filename = run_exp(
                    num_tries=num_tries,
                    alpha_airlines_bidder=alpha_airlines_bidder,
                    beta_resort_bidder=beta_resort_bidder,
                fileprefix=f"{alpha_strategy.__name__}_same_strategy__competing_companies_{alpha_budget}_budget_alpha_{beta_budget}_budget_beta")
                counter += 1
                print(f"Finished experiment {counter}/{experiment_count} with setting {alpha_strategy.__name__},{alpha_bid},{beta_bid},{alpha_strategy.__name__},{beta_budget},{alpha_budget}")